In [9]:
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && git pull && make clean && make GGML_CUDA=1
#!pip install -r llama.cpp/requirements.txt

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    

In [10]:
%%capture
!pip install rapidfuzz==3.8.1
!pip install langchain-community==0.2.7
!pip install pypdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os, time

# Download GGUF Llama-3-8b-Instruct GGUF 
Let's download the QuantFactory Q8_0 gguf version from HF and an example data to test the solution

In [12]:
!wget https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct.Q8_0.gguf

--2024-07-15 16:30:10--  https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct.Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 108.138.192.59, 108.138.192.82, 108.138.192.48, ...
Connecting to huggingface.co (huggingface.co)|108.138.192.59|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/79/f2/79f21025e377180e4ec0e3968bca4612bb9c99fa84e70cb7815186c42a858124/6f59b076f0e5f8456fe926f1c28580efd6c50dbf47d4b3fef9890ff8aee81c6b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Meta-Llama-3-8B-Instruct.Q8_0.gguf%3B+filename%3D%22Meta-Llama-3-8B-Instruct.Q8_0.gguf%22%3B&Expires=1721313010&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMTMxMzAxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc5L2YyLzc5ZjIxMDI1ZTM3NzE4MGU0ZWMwZTM5NjhiY2E0NjEyYmI5Yzk5ZmE4NGU3MGNiNzgxNTE4NmM0MmE4NTgxMjQvNmY1OWIwNzZmMGU1Zjg0NTZmZTkyNmYxYzI4NTgwZWZkNmM1MGRiZjQ3ZDRiM2ZlZjk4OTBmZjhhZWU4MWM2Yj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=DpSZDKon%7Ehsjspv1AFERbjO9izouL8fLUlVFsnNFwEdtcS9JQnwnsN3nJSTAFy127aCJn5ykAIObaqo8UKXhkdCH77sEUUYIwQh5oni%7EfvrrmfsPFu5t8WJuTRjHifiyErnHPpVGVX5O7Ey7lA7owt9EPFao%7ENZt5WWz4X24e%7E5WSNeMKIY%7EhHdjfCsyE2aa3s-J05qrIu9k

In [22]:
!wget https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/META-Q1-2024-Earnings-Call-Transcript.pdf

--2024-07-15 17:00:51--  https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/META-Q1-2024-Earnings-Call-Transcript.pdf
Resolving s21.q4cdn.com (s21.q4cdn.com)... 95.141.35.97
Connecting to s21.q4cdn.com (s21.q4cdn.com)|95.141.35.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238558 (233K) [application/pdf]
Saving to: ‘META-Q1-2024-Earnings-Call-Transcript.pdf’

META-Q1-2024-Earnin 100%[===================>] 232.97K  --.-KB/s    in 0.06s   

2024-07-15 17:00:51 (4.05 MB/s) - ‘META-Q1-2024-Earnings-Call-Transcript.pdf’ saved [238558/238558]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from langchain_community.document_loaders import PyPDFLoader

In [25]:
loader = PyPDFLoader("META-Q1-2024-Earnings-Call-Transcript.pdf")
data_pdf = loader.load()

In [46]:
data = "\n".join([d.page_content for d in data_pdf])
data[0:5000]

"1 \n Meta Platforms , Inc. ( META ) \nFirst Quarter 2024 Results Conference Call  \nApril 24th, 202 4 \n \nKen Dorell , Director , Investor Relations  \n \nThank you. Good afternoon and welcome to Meta Platforms first quarter 202 4 earnings conference call. \nJoining me today to discuss our results are Mark Zuckerberg, CEO and Susan Li, CFO.  \n \nBefore we get started, I would like to take this opportunity to remind you that our remarks today will \ninclude forward‐looking statements. Actual results may differ materially from those contemplated by \nthese forward‐looking statements.  \n \nFactors that could cause these results to differ materially are set forth in today’s earnings press release, \nand in our annual report on form 10 -K filed with the SEC. Any forward‐looking statements that we make \non this call are based on assumptions as of today and we undertake no obligation to update these \nstatements as a result of new information or future events.  \n \nDuring this call we w

In [39]:
import httpx
from transformers import AutoTokenizer
from langchain_text_splitters import RecursiveCharacterTextSplitter
import copy
from rapidfuzz import fuzz


class SyntheticDataGeneration:
    def __init__(self):   
        self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
    def add_message(self,role,message):
        if role not in ['system','user']:
            raise Exception("Not a valid role")
        self.messages.append({"role": role, "content": f"{message}"})
    def reset_history(self):
        self.messages = []
    async def make_request(self,prompt,num_tokens=250,temperature=0.3,top_k=1):
        async with httpx.AsyncClient() as client:
            # Define your JSON data
            json_data = {"prompt": f"{prompt}",
                         "n_predict": num_tokens,
                         "temperature":temperature,
                         "stop":["<|eot_id|>"],
                         "top_k":top_k}
            headers = {"Content-Type": "application/json"}
            # Make a POST request with JSON body
            response = await client.post('http://localhost:8888/completion', json=json_data,headers=headers,timeout=None)
            return response
    def format_prompt(self,user_message):
        self.add_message(role="user",message=user_message)
        prompt = self.tokenizer.apply_chat_template(
            self.messages,
            tokenize=False,
            add_generation_prompt=True
        )
        return prompt
    async def invoke(self,prompt,num_tokens=128):
        result = await self.make_request(prompt,num_tokens)
        return result
    async def check_question(self,question,num_tokens=500):
        base_instruction_prompt = """You are an assistant that help understand if a provided text is a valid question or not. Just return:
'VALID' if the text provided is a valid question
NOT_VALID if the text provided is NOT a valid question"""
        self.messages = [
            {"role": "system", "content": f"{base_instruction_prompt}"}
        ]
        p = f"""Question: {question}"""
        prompt = self.format_prompt(p)
        try:
            r = await self.invoke(prompt,num_tokens=num_tokens)
            if r.status_code==200:
                out = r.json()['content']
                if fuzz.ratio(out.lower(), "valid")>fuzz.ratio(out.lower(), "not_valid"):
                    return True
                else:
                    return False
                #questions = [q for q in questions if len(q)>=10]
            else:
                return None
        except Exception as e:
            print(repr(e))
            return None
       
    async def generate_questions(self,context,num_questions=5,num_tokens=500):
        base_instruction_prompt = """You are an assistant that help user generate questions about a provided context. Do not generate questions that cannot be answered by the information provided in the 'context'.
Do not provide anything else then questions.
Do not answer the questions that you generate.
Follow the structure below to generate the output:
Questions:
* How old are you?
* What are your favorite food?
* Have you ever been in USA?"""
        self.messages = [
            {"role": "system", "content": f"{base_instruction_prompt}"}
        ]
        p = f"""Generate a maximum of {num_questions} valid questions for the following Context. Do not provide anything then the questions. 
Context:
{context}"""
        prompt = self.format_prompt(p)
        prompt += "Questions:\n"
        try:
            r = await self.invoke(prompt,num_tokens=num_tokens)
            if r.status_code==200:
                questions = r.json()['content']
                qs_splitted = questions.split("*")
                questions = [q for q in qs_splitted if len(q)>=10]
            else:
                questions=None
        except Exception as e:
            print(repr(e))
            questions = None
        return questions
    async def answer_questions(self,context,question,num_tokens=500):
        base_instruction_prompt = f"""Given the following context generate a precise and detailed answer to the given question. Do not answer if the answer is not in the context provided.
Do not provide anything else then answer.
Context: {context}"""
        self.messages = [
            {"role": "system", "content": f"{base_instruction_prompt}"}
        ]
        p = f"Question:{question}"
        prompt = self.format_prompt(p)
        prompt += "Answer:"
        try:
            r = await self.invoke(prompt,num_tokens=num_tokens)
            if r.status_code==200:
                answer = r.json()['content']
            else:
                answer=None
        except Exception as e:
            answer = None
            print(repr(e))
        return answer
    def len(self,text):
        return len(self.tokenizer.encode(text))

    def split(self,content,chunk_size=250,chunk_overlap=25):
        text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=self.len,
            separators=[
            "\n\n",
            "\n",
            ".",
            " ",
            ",",
            "\u200B",  # Zero-width space
            "\uff0c",  # Fullwidth comma
            "\u3001",  # Ideographic comma
            "\uff0e",  # Fullwidth full stop
            "\u3002",  # Ideographic full stop
            "",
        ])
        texts = text_splitter.create_documents([content])
        return texts
    async def generate_dataset(self,text):
        texts = self.split(content=text,chunk_size=300,chunk_overlap=50)
        questions = []
        for t in texts:
            qs = await self.generate_questions(context=t.page_content,num_questions=10)
            print(qs)
            for qi in qs:
                questions.append({"question":qi,"context":t.page_content})
            self.reset_history()
        validated = []
        for q in questions:
            v = await self.check_question(question=q['question'])
            if v:
                validated.append(q)
        for v in validated:
            answer = await self.answer_questions(question=v['question'],context=v['context'],num_tokens=200)
            v['answer'] = answer
        return validated

# Run Llama.cpp Server
Use the following command in your terminal

```
./llama.cpp/llama-server -m Meta-Llama-3-8B-Instruct.Q8_0.gguf --port 8888 -ngl 50
```

In [40]:
synthetic_data_generator = SyntheticDataGeneration()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Let's run on 5000 chars to test it

In [41]:
dataset = await synthetic_data_generator.generate_dataset(text=data[0:5000])

[' Who is the CEO of Meta Platforms, Inc.?\n', ' What is the date of the conference call?\n', ' Who is the CFO of Meta Platforms, Inc.?\n', ' What type of statements will be included in the remarks during the call?\n', ' What is the purpose of the call?\n', ' What is the name of the person leading the call?\n', ' What is the name of the company being discussed?\n', ' What is the form number of the annual report filed with the SEC?\n', ' What type of financial measures will be presented during the call?']
[' What financial measures will be presented during the call?\n', ' Where can you find the reconciliation of GAAP to non-GAAP measures?\n', ' What is the purpose of the accompanying investor presentation?\n', " How many people use at least one of the company's apps each day?\n", " What is the growth rate of the company's business performance?\n", ' Which app is gaining momentum in terms of daily actives and message sends in the US?\n', " What are the company's AI efforts focused on?\n"

In [43]:
len(dataset)

54

In [44]:
dataset[0]

{'question': ' Who is the CEO of Meta Platforms, Inc.?\n',
 'context': '1 \n Meta Platforms , Inc. ( META ) \nFirst Quarter 2024 Results Conference Call  \nApril 24th, 202 4 \n \nKen Dorell , Director , Investor Relations  \n \nThank you. Good afternoon and welcome to Meta Platforms first quarter 202 4 earnings conference call. \nJoining me today to discuss our results are Mark Zuckerberg, CEO and Susan Li, CFO.  \n \nBefore we get started, I would like to take this opportunity to remind you that our remarks today will \ninclude forward‐looking statements. Actual results may differ materially from those contemplated by \nthese forward‐looking statements.  \n \nFactors that could cause these results to differ materially are set forth in today’s earnings press release, \nand in our annual report on form 10 -K filed with the SEC. Any forward‐looking statements that we make \non this call are based on assumptions as of today and we undertake no obligation to update these \nstatements as a 

In [45]:
import json
with open('dataset.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [47]:
dataset[0:5]

[{'question': ' Who is the CEO of Meta Platforms, Inc.?\n',
  'context': '1 \n Meta Platforms , Inc. ( META ) \nFirst Quarter 2024 Results Conference Call  \nApril 24th, 202 4 \n \nKen Dorell , Director , Investor Relations  \n \nThank you. Good afternoon and welcome to Meta Platforms first quarter 202 4 earnings conference call. \nJoining me today to discuss our results are Mark Zuckerberg, CEO and Susan Li, CFO.  \n \nBefore we get started, I would like to take this opportunity to remind you that our remarks today will \ninclude forward‐looking statements. Actual results may differ materially from those contemplated by \nthese forward‐looking statements.  \n \nFactors that could cause these results to differ materially are set forth in today’s earnings press release, \nand in our annual report on form 10 -K filed with the SEC. Any forward‐looking statements that we make \non this call are based on assumptions as of today and we undertake no obligation to update these \nstatements as 

### Great, starting from our files we can now create a great synthetic QA pairs with the relative chunk. 
### This can be used in the finetuning process to make our RAG base model better on our data, reduce errors and give the model a specific style. 